<a href="https://colab.research.google.com/github/kperv/summarizer_app/blob/main/KMeans_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarization project

for ods.ai *Natural Language Processing course*


## Clustering algorithm on aggregated word vectors.


*   Supported languages are Russian and Spanish.
*   Text is broken into sentences by Spacy.
*   Sentences are tokenized by pretrained BertTokenizer.
*   sklearn is used for KMeans Clustering
*   Evaluation metrics are bert-score and rouge


## Installations and imports

In [1]:
%%capture
!pip install transformers[sentencepiece]
!pip install pytorch-lightning
!pip install -U bert_score
!pip install datasets
!pip install https://huggingface.co/spacy/ru_core_news_md/resolve/main/ru_core_news_md-any-py3-none-any.whl
!pip install https://huggingface.co/spacy/es_core_news_md/resolve/main/es_core_news_md-any-py3-none-any.whl


import os
import numpy as np
import spacy
import transformers
import datasets
import bert_score
import sklearn
import pandas as pd
from rouge import Rouge

from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from transformers import BertTokenizer, BertModel
from bert_score import score

In [2]:
nlp_ru = spacy.load("ru_core_news_md")
nlp_es = spacy.load("es_core_news_md")
nlp = {"ru": nlp_ru, "es": nlp_es}

### requirements

In [3]:
print("numpy=={}".format(np.__version__))
print("spacy=={}".format(spacy.__version__))
print("transformers=={}".format(transformers.__version__))
print("bert_score=={}".format(bert_score.__version__))
print("sklearn=={}".format(sklearn.__version__))

numpy==1.19.5
spacy==3.2.1
transformers==4.13.0
bert_score==0.3.11
sklearn==1.0.1


### Examples and checks of Bert score

Predictions correspond to a news headline and references to a ferst paragraph of the same article.

In [4]:
predictions = ["Аналитик прокомментировал предстоящий визит Меркель в Москву"]
references = ["Ведущий научный сотрудник Центра германских исследований Института Европы РАН Александр Камкин прокомментировал в беседе с RT сообщение о том, что канцлер Германии Ангела Меркель и президент России Владимир Путин проведут переговоры в Москве 20 августа"]
P, R, F1 = score(predictions, references, lang='ru')
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level P score: {P.mean():.3f}")
print(f"System level R score: {R.mean():.3f}")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

System level F1 score: 0.680
System level P score: 0.738
System level R score: 0.632


In [5]:
predictions = ["Qué sabemos sobre el diálogo entre la oposición y Maduro que está programado para comenzar este viernes en México"]
references = ["Tras años de tensiones, protestas y negociaciones estancadas, en medio de una situación económica muy deteriorada y complicada aún más por la pandemia de covid-19, el gobierno de Venezuela y la oposición intentará por quinta vez llegar a una solución para la crisis política mediante un diálogo, esta vez en México."]
P, R, F1 = score(predictions, references, lang='es')
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level P score: {P.mean():.3f}")
print(f"System level R score: {R.mean():.3f}")

System level F1 score: 0.636
System level P score: 0.663
System level R score: 0.611


## Extractive text summarization

### data

In [6]:
number = 3
lang = "ru"

Some random paragraph from news

In [7]:
text = "Очень непросто делать прогнозы. С одной стороны, достаточно долго длится фаза подъема, после 11 июня началась третья волна в Свердловской области. Мы достигли очень высокого уровня заболеваемости и смертности и стабилизировались на нем. Как долго это будет продолжаться, зависит от доли восприимчивого к вирусу населения. На момент начала третьей волны число привитых или переболевших свердловчан не превышало 50 процентов, — отметил Соловьев. — Чтобы третья волна остановилась, мы должны достичь высокого уровня коллективного иммунитета. С помощью вакцинации мы уже не успеваем его достичь. Второй вариант — заболеваемость. Официальная статистика не отражает реальное число людей, которые встретились с коронавирусом. Сейчас приблизительно 65% жителей переболели или вакцинировались от коронавируса. Это должно сказываться на снижении заболеваемости, но пока этого не происходит."

### Separate sentences with SpaCy

In [8]:
def break_text(text, lang):
  doc = nlp[lang](text)
  assert doc.has_annotation("SENT_START")
  sentences = [str(sent) for sent in doc.sents]
  return sentences

In [9]:
sentences = break_text(text, lang)

### Convert words to tokens and run through Bert encoder to get word embeddings

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
encoded_sentences = tokenizer(sentences, truncation=True, padding=True, return_tensors="pt")
outputs = model(encoded_sentences.input_ids)
embeddings = outputs.last_hidden_state

I'm using *mean* to get sentence embeddings. This step can be improved.

In [11]:
embeddings = embeddings.detach().numpy()
embeddings = embeddings.mean(axis=1)

Cluster embeddings in a space into n (number) clusters to get centroids. Take distance metric between sentence embeddings and centroids and choose n closest to put into the summary.

### Collect summary sentences together

In [12]:
kmeans = KMeans(n_clusters=number).fit(embeddings)
centroids = kmeans.cluster_centers_
result = pairwise_distances_argmin_min(embeddings, centroids)
summary_sent_positions = list(np.argsort(result[1])[:number])

In [13]:
summary = ""
for idx in summary_sent_positions:
  summary += sentences[idx]
summary

'На момент начала третьей волны число привитых или переболевших свердловчан не превышало 50 процентов, — отметил Соловьев.Очень непросто делать прогнозы.Второй вариант — заболеваемость.'

### Calculate Bert-score

In [14]:
predictions = [summary]
references = [text]
P, R, F1 = score(predictions, references, lang=lang)
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level P score: {P.mean():.3f}")
print(f"System level R score: {R.mean():.3f}")

System level F1 score: 0.770
System level P score: 0.898
System level R score: 0.675


## Evaluate solution on 200 samples

In [16]:
! git clone https://github.com/kperv/summarizer_app.git
! pip install -r summarizer_app/requirements.txt

Cloning into 'summarizer_app'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 157 (delta 61), reused 96 (delta 24), pack-reused 0
Receiving objects: 100% (157/157), 3.23 MiB | 5.94 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [18]:
def collect_modified_dataset(data_path):
    transformed_df = pd.DataFrame()
    for dataset_part in os.listdir(data_path):
        part_path = os.path.join(data_path, dataset_part)
        df_slice = pd.read_csv(part_path)
        transformed_df = pd.concat([transformed_df, df_slice])
    return transformed_df

In [24]:
data_dir = os.getcwd()
data_path = data_dir + "/summarizer_app/data"
df = collect_modified_dataset(data_path)

,Unnamed: 0,text,summary
0,50,Как рассказали “МК” в СКП по Московской област...,рублей. Незадолго до похищения паренек рассказ...
1,51,"Как стало известно “МК”, головокружительный по...","Судя по диктофонной записи из кабины, друзья е..."
2,52,А под Новый год подросшего малыша приняла обра...,Малышка оказалась здоровой и спокойной. Но общ...


In [29]:
def get_rouge_score(sample):
    rouge = Rouge()
    preprocess_exs = lambda exs : [ex.strip().lower() for ex in exs]
    predictions = []
    predictions.append(sample['summary'])
    predictions = preprocess_exs(predictions)
    references = []
    references.append(sample.text)
    references = preprocess_exs(references)
    predictions = [pred if len(pred) else 'а' for pred in predictions]
    rouge_scores =  rouge.get_scores(predictions, references, avg=True)
    return {k: round(v['f'], 3) for k, v in rouge_scores.items()}

In [30]:
def add_metrics(dataset):
    dataset = dataset.loc[:, ['text', 'summary']]
    dataset[['rouge-1', 'rouge-2', 'rouge-l']] = 0, 0, 0
    df = pd.DataFrame(list(dataset.apply(get_rouge_score, axis=1).values))
    dataset = df.combine_first(dataset)
    dataset = dataset.reindex(
        columns=['text', 'summary', 'rouge-1', 'rouge-2', 'rouge-l']
    )
    return dataset

In [31]:
df = add_metrics(df)
df.head(3)

,text,summary,rouge-1,rouge-2,rouge-l
0,Как рассказали “МК” в СКП по Московской област...,рублей. Незадолго до похищения паренек рассказ...,0.259,0.195,0.152
0,Сладострастник в течение трех лет преследовал ...,После этого жизнь мальчика превратилась в сущи...,0.259,0.195,0.152
0,"Не исключено, что в ближайшие же дни многие ро...",Заклеенные лентой разрывы на бумажных деньгах ...,0.259,0.195,0.152


In [32]:
round(df['rouge-1'].mean(), 3)

0.359

In [33]:
round(df['rouge-2'].mean(), 3)

0.312

In [34]:
round(df['rouge-l'].mean(), 3)

0.267